In [1]:
import os
import json
from glob import glob

In [2]:
import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)


def sqlite_main():
    database = r"top.db"

    sql_create_images_table = """ CREATE TABLE IF NOT EXISTS images (
                                        id integer PRIMARY KEY,
                                        dataset_name text NOT NULL,
                                        image_name text NOT NULL,
                                        rel_flag BOOLEAN NOT NULL CHECK (rel_flag IN (0,1))
                                    ); """

    sql_create_objects_table = """CREATE TABLE IF NOT EXISTS objects (
                                    id integer PRIMARY KEY,
                                    object_name text NOT NULL,
                                    score real NOT NULL,
                                    dataset_name text NOT NULL,
                                    image_name text NOT NULL,
                                    image_id integer NOT NULL,
                                    FOREIGN KEY (image_id) REFERENCES images (id)
                                );"""
    

    sql_create_relations_table = """CREATE TABLE IF NOT EXISTS relations (
                                    id integer PRIMARY KEY,
                                    object_1 text NOT NULL,
                                    relation text NOT NULL,
                                    object_2 text NOT NULL,
                                    score real NOT NULL,
                                    dataset_name text NOT NULL,
                                    image_name text NOT NULL,
                                    image_id integer NOT NULL,
                                    FOREIGN KEY (image_id) REFERENCES images (id)
                                );"""


    # create a database connection
    conn = create_connection(database)
    
    # create tables
    if conn is not None:
        # create images table
        create_table(conn, sql_create_images_table)
    
        # create objects table
        create_table(conn, sql_create_objects_table)
    
        # create relations table
        create_table(conn, sql_create_relations_table)
    else:
        print("Error! cannot create the database connection.")

sqlite_main()

In [3]:
import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn


def insert_image(conn, image):
    """
    Create a new project into the projects table
    :param conn:
    :param image:
    :return: image id
    """
    sql = ''' INSERT INTO images(dataset_name, image_name,rel_flag)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, image)
    conn.commit()
    return cur.lastrowid


def insert_object(conn, _object):
    """
    Create a new object
    :param conn:
    :param object:
    :return:
    """

    sql = ''' INSERT INTO objects(object_name,score,dataset_name,image_name,image_id)
              VALUES(?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, _object)
    conn.commit()
    return cur.lastrowid


def insert_relation(conn, _rel):
    """
    Create a new relation
    :param conn:
    :param _rel:
    :return:
    """

    sql = ''' INSERT INTO relations(object_1,relation,object_2,score,dataset_name,image_name,image_id)
              VALUES(?,?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, _rel)
    conn.commit()
    return cur.lastrowid



In [4]:
def insert_data(data_dir, e_check, dataset='Eurocity'):
    database = r"top.db"
    # create a database connection
    conn = create_connection(database)
    
    box_topk = 30  # select top k bounding boxes
    rel_topk = 20  # select top k relationships

    for d in data_dir:

        print(f"we are trying to filter prediction for {d}")

        info_path = os.path.join(e_check, d, 'custom_data_info.json')
        prediction_file = os.path.join(e_check, d, 'custom_prediction.json')
        custom_prediction = json.load(open(prediction_file))
        custom_data_info = json.load(open(info_path))

        ind_to_classes = custom_data_info['ind_to_classes']
        ind_to_predicates = custom_data_info['ind_to_predicates']

        human_labels = ['woman', 'men', 'boy', 'kid', 'child', 'guy', 'man', 'person', 'girl', 'lady', 'people']

        for image_idx in range(len(custom_data_info['idx_to_files'])):
            file_name, _ = os.path.basename(custom_data_info['idx_to_files'][image_idx]).rsplit(".")
            image_path = custom_data_info['idx_to_files'][image_idx]
            boxes = custom_prediction[str(image_idx)]['bbox'][:box_topk]
            box_labels = custom_prediction[str(image_idx)]['bbox_labels'][:box_topk]
            box_scores = custom_prediction[str(image_idx)]['bbox_scores'][:box_topk]
            all_rel_labels = custom_prediction[str(image_idx)]['rel_labels']
            all_rel_scores = custom_prediction[str(image_idx)]['rel_scores']
            all_rel_pairs = custom_prediction[str(image_idx)]['rel_pairs']

            for i in range(len(box_labels)):
                box_labels[i] = ind_to_classes[box_labels[i]]

            rel_labels_list = []
            rel_scores = []

            new_rel_pairs = []
            new_rel_labels = []
            new_rel_scores = []

            for i in range(len(all_rel_pairs)):
                if all_rel_pairs[i][0] < box_topk and all_rel_pairs[i][1] < box_topk:
                    # if box_labels[all_rel_pairs[i][0]] in human_labels or box_labels[all_rel_pairs[i][1]] in human_labels:
                    new_rel_pairs.append(all_rel_pairs[i])
                    new_rel_labels.append(all_rel_labels[i])
                    new_rel_scores.append(all_rel_scores[i])

            h_flag = False
            new_rel_scores = new_rel_scores[:rel_topk]
            new_rel_pairs = new_rel_pairs[:rel_topk]
            new_rel_labels = new_rel_labels[:rel_topk]
            for i in range(len(new_rel_pairs)):
                if new_rel_pairs[i][0] < box_topk and new_rel_pairs[i][1] < box_topk:
                    if box_labels[new_rel_pairs[i][0]] in human_labels or box_labels[new_rel_pairs[i][1]] in human_labels:
                        h_flag = True
                    r_list = [box_labels[new_rel_pairs[i][0]], ind_to_predicates[new_rel_labels[i]],
                              box_labels[new_rel_pairs[i][1]]]
                    rel_labels_list.append(r_list)

#             print(rel_labels_list)
#             print(new_rel_scores)
#             print(box_labels)
#             print(box_scores)
            if h_flag:
                rel_flag = '1'
            else:
                rel_flag = '0'

            image_ = (dataset, file_name, rel_flag)
            image_id = insert_image(conn, image_)

            for idx, _obj in enumerate(box_labels):
                obj_tuples = (_obj, box_scores[idx], dataset, file_name, image_id)
                insert_object(conn, obj_tuples)

            # relations
            for idx, rel in enumerate(rel_labels_list):
                rel_ = (rel[0], rel[1], rel[2], new_rel_scores[idx], dataset, file_name, image_id)

                insert_relation(conn, rel_)
#             break
#         break

In [19]:
e_check = "/home/test/Scene-Graph/Scene/checkpoint/eurocity"

e_image_dir = glob(f"{e_check}/*")
train_dir_name = [x.rsplit("/", 1)[1] for x in e_image_dir]
print(train_dir_name)

insert_data(train_dir_name, e_check, dataset='Eurocity')


['ljubljana_day', 'potsdam_day', 'marseille_day', 'hamburg_day', 'firenze_day', 'nuernberg_day', 'dresden_day', 'szczecin_day', 'amsterdam_day', 'brno_day', 'barcelona_day', 'roma_day', 'prague_day', 'budapest_day', 'stuttgart_day', 'pisa_day', 'wuerzburg_day', 'zagreb_day', 'torino_day', 'berlin_day', 'zuerich_day', 'toulouse_day', 'lyon_day', 'koeln_day', 'milano_day', 'montpellier_day', 'leipzig_day', 'ulm_day', 'bratislava_day', 'basel_day', 'bologna_day']
we are trying to filter prediction for ljubljana_day
we are trying to filter prediction for potsdam_day
we are trying to filter prediction for marseille_day
we are trying to filter prediction for hamburg_day
we are trying to filter prediction for firenze_day
we are trying to filter prediction for nuernberg_day
we are trying to filter prediction for dresden_day
we are trying to filter prediction for szczecin_day
we are trying to filter prediction for amsterdam_day
we are trying to filter prediction for brno_day
we are trying to fi

In [5]:
e_check = "/home/test/Scene-Graph/Scene/checkpoint/coco"

e_image_dir = glob(f"{e_check}/*")
train_dir_name = [x.rsplit("/", 1)[1] for x in e_image_dir]
print(train_dir_name)

insert_data(train_dir_name, e_check, dataset='COCO')

['train_2', 'train_1', 'train_3']
we are trying to filter prediction for train_2
we are trying to filter prediction for train_3


In [21]:
! ls "/home/test/Scene-Graph/Scene/checkpoint/caltech_output"

train_1  train_2


In [22]:
e_check = "/home/test/Scene-Graph/Scene/checkpoint/caltech_output"

e_image_dir = glob(f"{e_check}/*")
train_dir_name = [x.rsplit("/", 1)[1] for x in e_image_dir]
print(train_dir_name)

insert_data(train_dir_name, e_check, ataset='Caltech')


['train_2', 'train_1']
we are trying to filter prediction for train_2
we are trying to filter prediction for train_1


In [24]:
! ls "/home/test/Scene-Graph/Scene/checkpoint/custom_output_path"

custom_data_info.json  custom_prediction.json


In [28]:
e_check = "/home/test/Scene-Graph/Scene/checkpoint"
train_dir_name = ["custom_output_path"]
insert_data(train_dir_name, e_check, dataset='CityPerson')

we are trying to filter prediction for custom_output_path


In [30]:
e_check = "/home/test/Scene-Graph/Scene/checkpoint/CrowdHuman"

e_image_dir = glob(f"{e_check}/*")
train_dir_name = [x.rsplit("/", 1)[1] for x in e_image_dir]
print(train_dir_name)

insert_data(train_dir_name, e_check, dataset='CrowdHuman')

['train_2', 'train_5', 'train_4', 'train_1', 'train_3']
we are trying to filter prediction for train_2
we are trying to filter prediction for train_5
we are trying to filter prediction for train_4
we are trying to filter prediction for train_1
we are trying to filter prediction for train_3


In [31]:
e_check = "/home/test/Scene-Graph/Scene/checkpoint/WiderPerson"

e_image_dir = glob(f"{e_check}/*")
train_dir_name = [x.rsplit("/", 1)[1] for x in e_image_dir]
print(train_dir_name)

insert_data(train_dir_name, e_check, dataset='WiderPerson')

['train_2', 'train_1', 'train_3']
we are trying to filter prediction for train_2
we are trying to filter prediction for train_1
we are trying to filter prediction for train_3


In [33]:
database = r"top.db"

# create a database connection
conn = create_connection(database)
cur = conn.cursor()


# cur.execute("SELECT * FROM relations")

cur.execute("SELECT * FROM images")
rows = cur.fetchall()

for row in rows:
    print(row)

(1, 'Eurocity', 'ljubljana_00599', 0)
(2, 'Eurocity', 'ljubljana_00667', 0)
(3, 'Eurocity', 'ljubljana_00009', 0)
(4, 'Eurocity', 'ljubljana_00828', 1)
(5, 'Eurocity', 'ljubljana_00872', 0)
(6, 'Eurocity', 'ljubljana_00642', 0)
(7, 'Eurocity', 'ljubljana_00102', 1)
(8, 'Eurocity', 'ljubljana_00688', 1)
(9, 'Eurocity', 'ljubljana_00226', 0)
(10, 'Eurocity', 'ljubljana_00562', 1)
(11, 'Eurocity', 'ljubljana_00547', 1)
(12, 'Eurocity', 'ljubljana_00833', 0)
(13, 'Eurocity', 'ljubljana_00724', 0)
(14, 'Eurocity', 'ljubljana_00393', 0)
(15, 'Eurocity', 'ljubljana_00013', 1)
(16, 'Eurocity', 'ljubljana_00765', 0)
(17, 'Eurocity', 'ljubljana_00279', 1)
(18, 'Eurocity', 'ljubljana_00795', 0)
(19, 'Eurocity', 'ljubljana_00192', 1)
(20, 'Eurocity', 'ljubljana_00387', 0)
(21, 'Eurocity', 'ljubljana_00693', 1)
(22, 'Eurocity', 'ljubljana_00353', 0)
(23, 'Eurocity', 'ljubljana_00302', 0)
(24, 'Eurocity', 'ljubljana_00260', 1)
(25, 'Eurocity', 'ljubljana_00177', 0)
(26, 'Eurocity', 'ljubljana_00366'

(1792, 'Eurocity', 'marseille_00347', 1)
(1793, 'Eurocity', 'marseille_00012', 1)
(1794, 'Eurocity', 'marseille_01054', 1)
(1795, 'Eurocity', 'marseille_01140', 0)
(1796, 'Eurocity', 'marseille_01156', 1)
(1797, 'Eurocity', 'marseille_00326', 0)
(1798, 'Eurocity', 'marseille_01161', 0)
(1799, 'Eurocity', 'marseille_01064', 1)
(1800, 'Eurocity', 'marseille_01098', 1)
(1801, 'Eurocity', 'marseille_00841', 1)
(1802, 'Eurocity', 'marseille_01174', 1)
(1803, 'Eurocity', 'marseille_01134', 0)
(1804, 'Eurocity', 'marseille_00102', 1)
(1805, 'Eurocity', 'marseille_01015', 0)
(1806, 'Eurocity', 'marseille_00586', 1)
(1807, 'Eurocity', 'marseille_00561', 1)
(1808, 'Eurocity', 'marseille_00352', 1)
(1809, 'Eurocity', 'marseille_00161', 1)
(1810, 'Eurocity', 'marseille_00275', 1)
(1811, 'Eurocity', 'marseille_00480', 1)
(1812, 'Eurocity', 'marseille_00843', 0)
(1813, 'Eurocity', 'marseille_00065', 1)
(1814, 'Eurocity', 'marseille_00452', 1)
(1815, 'Eurocity', 'marseille_00980', 0)
(1816, 'Eurocity

(3525, 'Eurocity', 'firenze_00775', 1)
(3526, 'Eurocity', 'firenze_00641', 0)
(3527, 'Eurocity', 'firenze_00049', 0)
(3528, 'Eurocity', 'firenze_00277', 1)
(3529, 'Eurocity', 'firenze_01051', 1)
(3530, 'Eurocity', 'firenze_00499', 1)
(3531, 'Eurocity', 'firenze_00356', 0)
(3532, 'Eurocity', 'firenze_00648', 0)
(3533, 'Eurocity', 'firenze_00829', 1)
(3534, 'Eurocity', 'firenze_00239', 1)
(3535, 'Eurocity', 'firenze_00714', 0)
(3536, 'Eurocity', 'firenze_00618', 1)
(3537, 'Eurocity', 'firenze_00054', 1)
(3538, 'Eurocity', 'firenze_00379', 0)
(3539, 'Eurocity', 'firenze_01087', 0)
(3540, 'Eurocity', 'firenze_01097', 1)
(3541, 'Eurocity', 'firenze_00483', 1)
(3542, 'Eurocity', 'firenze_00821', 0)
(3543, 'Eurocity', 'firenze_00907', 0)
(3544, 'Eurocity', 'firenze_00407', 1)
(3545, 'Eurocity', 'firenze_01046', 1)
(3546, 'Eurocity', 'firenze_00983', 1)
(3547, 'Eurocity', 'firenze_00439', 1)
(3548, 'Eurocity', 'firenze_00902', 1)
(3549, 'Eurocity', 'firenze_00888', 1)
(3550, 'Eurocity', 'firen

(5339, 'Eurocity', 'nuernberg_00125', 0)
(5340, 'Eurocity', 'nuernberg_00612', 0)
(5341, 'Eurocity', 'nuernberg_00584', 1)
(5342, 'Eurocity', 'nuernberg_00342', 0)
(5343, 'Eurocity', 'nuernberg_00427', 0)
(5344, 'Eurocity', 'nuernberg_00174', 1)
(5345, 'Eurocity', 'nuernberg_00333', 1)
(5346, 'Eurocity', 'nuernberg_00602', 0)
(5347, 'Eurocity', 'nuernberg_00015', 0)
(5348, 'Eurocity', 'nuernberg_00829', 0)
(5349, 'Eurocity', 'nuernberg_00207', 1)
(5350, 'Eurocity', 'nuernberg_00330', 0)
(5351, 'Eurocity', 'nuernberg_00154', 1)
(5352, 'Eurocity', 'nuernberg_00034', 0)
(5353, 'Eurocity', 'nuernberg_00757', 1)
(5354, 'Eurocity', 'nuernberg_00857', 1)
(5355, 'Eurocity', 'nuernberg_00656', 1)
(5356, 'Eurocity', 'nuernberg_00728', 1)
(5357, 'Eurocity', 'nuernberg_00016', 0)
(5358, 'Eurocity', 'nuernberg_00618', 1)
(5359, 'Eurocity', 'nuernberg_00373', 1)
(5360, 'Eurocity', 'nuernberg_00075', 1)
(5361, 'Eurocity', 'nuernberg_00118', 0)
(5362, 'Eurocity', 'nuernberg_00772', 0)
(5363, 'Eurocity

(7044, 'Eurocity', 'amsterdam_00453', 1)
(7045, 'Eurocity', 'amsterdam_00219', 0)
(7046, 'Eurocity', 'amsterdam_00371', 1)
(7047, 'Eurocity', 'amsterdam_01035', 1)
(7048, 'Eurocity', 'amsterdam_00390', 1)
(7049, 'Eurocity', 'amsterdam_00084', 0)
(7050, 'Eurocity', 'amsterdam_00877', 0)
(7051, 'Eurocity', 'amsterdam_00048', 1)
(7052, 'Eurocity', 'amsterdam_00656', 1)
(7053, 'Eurocity', 'amsterdam_00129', 0)
(7054, 'Eurocity', 'amsterdam_00172', 0)
(7055, 'Eurocity', 'amsterdam_00293', 1)
(7056, 'Eurocity', 'amsterdam_00142', 0)
(7057, 'Eurocity', 'amsterdam_01015', 1)
(7058, 'Eurocity', 'amsterdam_00432', 1)
(7059, 'Eurocity', 'amsterdam_00898', 0)
(7060, 'Eurocity', 'amsterdam_00109', 1)
(7061, 'Eurocity', 'amsterdam_00001', 0)
(7062, 'Eurocity', 'amsterdam_00363', 1)
(7063, 'Eurocity', 'amsterdam_00262', 1)
(7064, 'Eurocity', 'amsterdam_01039', 0)
(7065, 'Eurocity', 'amsterdam_00747', 0)
(7066, 'Eurocity', 'amsterdam_00440', 0)
(7067, 'Eurocity', 'amsterdam_00120', 0)
(7068, 'Eurocity

...

(46290, 'WiderPerson', '000482', 1)
(46291, 'WiderPerson', '001736', 1)
(46292, 'WiderPerson', '018435', 1)
(46293, 'WiderPerson', '013123', 1)
(46294, 'WiderPerson', '007433', 1)
(46295, 'WiderPerson', '006823', 1)
(46296, 'WiderPerson', '011257', 1)
(46297, 'WiderPerson', '001069', 1)
(46298, 'WiderPerson', '014939', 1)
(46299, 'WiderPerson', '003597', 1)
(46300, 'WiderPerson', '015183', 1)
(46301, 'WiderPerson', '010792', 1)
(46302, 'WiderPerson', '008005', 1)
(46303, 'WiderPerson', '000637', 1)
(46304, 'WiderPerson', '015890', 1)
(46305, 'WiderPerson', '014043', 1)
(46306, 'WiderPerson', '008769', 1)
(46307, 'WiderPerson', '007996', 1)
(46308, 'WiderPerson', '012541', 1)
(46309, 'WiderPerson', '005783', 1)
(46310, 'WiderPerson', '010318', 1)
(46311, 'WiderPerson', '019092', 1)
(46312, 'WiderPerson', '004614', 1)
(46313, 'WiderPerson', '016901', 1)
(46314, 'WiderPerson', '013035', 1)
(46315, 'WiderPerson', '005610', 1)
(46316, 'WiderPerson', '003523', 1)
(46317, 'WiderPerson', '0006

(48104, 'WiderPerson', '017696', 1)
(48105, 'WiderPerson', '020107', 1)
(48106, 'WiderPerson', '013683', 1)
(48107, 'WiderPerson', '016682', 1)
(48108, 'WiderPerson', '019676', 1)
(48109, 'WiderPerson', '004084', 1)
(48110, 'WiderPerson', '000976', 1)
(48111, 'WiderPerson', '004597', 1)
(48112, 'WiderPerson', '015671', 1)
(48113, 'WiderPerson', '013754', 1)
(48114, 'WiderPerson', '003782', 1)
(48115, 'WiderPerson', '015706', 1)
(48116, 'WiderPerson', '014459', 1)
(48117, 'WiderPerson', '001029', 1)
(48118, 'WiderPerson', '005019', 1)
(48119, 'WiderPerson', '008203', 1)
(48120, 'WiderPerson', '010769', 1)
(48121, 'WiderPerson', '020399', 1)
(48122, 'WiderPerson', '005920', 1)
(48123, 'WiderPerson', '017949', 1)
(48124, 'WiderPerson', '007034', 1)
(48125, 'WiderPerson', '019630', 1)
(48126, 'WiderPerson', '016255', 1)
(48127, 'WiderPerson', '005805', 1)
(48128, 'WiderPerson', '013999', 1)
(48129, 'WiderPerson', '010014', 1)
(48130, 'WiderPerson', '019864', 1)
(48131, 'WiderPerson', '0054

(49941, 'WiderPerson', '017153', 1)
(49942, 'WiderPerson', '013230', 1)
(49943, 'WiderPerson', '000767', 1)
(49944, 'WiderPerson', '018982', 1)
(49945, 'WiderPerson', '001631', 1)
(49946, 'WiderPerson', '011621', 1)
(49947, 'WiderPerson', '013829', 1)
(49948, 'WiderPerson', '011685', 1)
(49949, 'WiderPerson', '020142', 1)
(49950, 'WiderPerson', '012949', 1)
(49951, 'WiderPerson', '014528', 1)
(49952, 'WiderPerson', '003008', 1)
(49953, 'WiderPerson', '005125', 1)
(49954, 'WiderPerson', '006818', 1)
(49955, 'WiderPerson', '010097', 1)
(49956, 'WiderPerson', '018474', 1)
(49957, 'WiderPerson', '003306', 1)
(49958, 'WiderPerson', '006335', 1)
(49959, 'WiderPerson', '006166', 1)
(49960, 'WiderPerson', '001052', 1)
(49961, 'WiderPerson', '000832', 1)
(49962, 'WiderPerson', '018827', 1)
(49963, 'WiderPerson', '010898', 1)
(49964, 'WiderPerson', '019622', 1)
(49965, 'WiderPerson', '004690', 1)
(49966, 'WiderPerson', '005804', 1)
(49967, 'WiderPerson', '008246', 1)
(49968, 'WiderPerson', '0203

(51767, 'WiderPerson', '012651', 1)
(51768, 'WiderPerson', '018489', 1)
(51769, 'WiderPerson', '017414', 1)
(51770, 'WiderPerson', '010697', 1)
(51771, 'WiderPerson', '012129', 1)
(51772, 'WiderPerson', '007641', 1)
(51773, 'WiderPerson', '016621', 1)
(51774, 'WiderPerson', '003242', 1)
(51775, 'WiderPerson', '005358', 1)
(51776, 'WiderPerson', '005248', 1)
(51777, 'WiderPerson', '014124', 1)
(51778, 'WiderPerson', '001153', 1)
(51779, 'WiderPerson', '006910', 1)
(51780, 'WiderPerson', '003305', 1)
(51781, 'WiderPerson', '014468', 1)
(51782, 'WiderPerson', '020514', 1)
(51783, 'WiderPerson', '004139', 1)
(51784, 'WiderPerson', '001030', 1)
(51785, 'WiderPerson', '003157', 1)
(51786, 'WiderPerson', '016053', 1)
(51787, 'WiderPerson', '012664', 1)
(51788, 'WiderPerson', '013054', 1)
(51789, 'WiderPerson', '010766', 1)
(51790, 'WiderPerson', '005282', 1)
(51791, 'WiderPerson', '019585', 1)
(51792, 'WiderPerson', '001204', 1)
(51793, 'WiderPerson', '004504', 1)
(51794, 'WiderPerson', '0156

(53575, 'WiderPerson', '018214', 1)
(53576, 'WiderPerson', '015845', 1)
(53577, 'WiderPerson', '001531', 1)
(53578, 'WiderPerson', '006656', 1)
(53579, 'WiderPerson', '001489', 1)
(53580, 'WiderPerson', '014758', 1)
(53581, 'WiderPerson', '011232', 1)
(53582, 'WiderPerson', '014238', 1)
(53583, 'WiderPerson', '018471', 1)
(53584, 'WiderPerson', '012191', 1)
(53585, 'WiderPerson', '005987', 1)
(53586, 'WiderPerson', '011030', 1)
(53587, 'WiderPerson', '005892', 1)
(53588, 'WiderPerson', '016233', 1)
(53589, 'WiderPerson', '016999', 1)
(53590, 'WiderPerson', '018997', 1)
(53591, 'WiderPerson', '004780', 1)
(53592, 'WiderPerson', '009626', 1)
(53593, 'WiderPerson', '000431', 1)
(53594, 'WiderPerson', '009010', 1)
(53595, 'WiderPerson', '001612', 1)
(53596, 'WiderPerson', '018616', 1)
(53597, 'WiderPerson', '019696', 1)
(53598, 'WiderPerson', '015908', 1)
(53599, 'WiderPerson', '009527', 1)
(53600, 'WiderPerson', '004737', 1)
(53601, 'WiderPerson', '016847', 1)
(53602, 'WiderPerson', '0085